# HW3: Netflix Data Analysis

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import scipy.sparse
import random

In [2]:
ratings = scipy.sparse.load_npz('sample_matrix.npz')

In [3]:
test_value = np.load('test_value.npz')['arr_0']
test_movie = np.load('test_movie.npz')['arr_0']
test_user = np.load('test_user.npz')['arr_0']
train_value = np.load('train_value.npz')['arr_0']
train_movie = np.load('train_movie.npz')['arr_0']
train_user = np.load('train_user.npz')['arr_0']

In [7]:
movie_avg = np.zeros(5000)
user_avg = np.zeros (10000)
for i in range(5000):
    movie_avg[i] = csr_matrix.mean(ratings[i, :])
for i in range(10000):
    user_avg[i] =csr_matrix.mean(ratings[:, i])

In [13]:
#Movie avg, user avg, # movie reviews, # user reviews, # positive movie reviews, # negative movie review
# #positive user reiews, # negative user reviews
movie_avg_train = np.zeros(len(train_value))
user_avg_train = np.zeros(len(train_value))
for i in range(len(train_value)):
    movie_avg_train[i] = movie_avg[train_movie[i]]
    user_avg_train[i] = user_avg[train_user[i]]

features_train = np.vstack((movie_avg_train, user_avg_train)).T

movie_avg_test = np.zeros(len(test_value))
user_avg_test = np.zeros(len(test_value))
for i in range(len(test_value)):
    movie_avg_test[i] = movie_avg[test_movie[i]]
    user_avg_test[i] = user_avg[test_user[i]]

features_test = np.vstack((movie_avg_test, user_avg_test)).T

In [16]:
def MSE(predictions, test_set):
    summation = 0
    for i in range(len(predictions)):
        summation += (predictions[i] - test_set[i])**2
    return summation/len(predictions)

In [17]:
def MSE_non_zero(predictions, test_set):
    summation = 0
    non_zero = 0
    for i in range(len(predictions)):
        value = test_set[i]
        if value > 0:
            summation += (predictions[i] - value)**2
            non_zero += 1
    return summation/non_zero

In [18]:
print MSE(movie_avg_test, test_value)
print MSE(user_avg_test, test_value)
print MSE_non_zero(movie_avg_test, test_value)
print MSE_non_zero(user_avg_test, test_value)

0.0307532696129
0.0303709616256
13.796825834
13.4713849259


In [21]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(features_train, train_value)
predictions = lr.predict(features_test)

/usr/local/lib/python2.7/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [22]:
print MSE(predictions, test_value)
print MSE_non_zero(predictions, test_value)

0.0297568250505
12.9444626023
